In [1]:
"""Manejo de Informacion"""

import pandas as pd
import numpy as np
#from pandas import json_normalize
#import requests
#import json

#"""Tiempo"""

#from datetime import datetime
#from datetime import timezone

"""Textos"""

import re 
from unidecode import unidecode
import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from itertools import chain
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from nltk.stem import SnowballStemmer
from wordcloud import WordCloud
from collections import Counter

"""Visualizaciones"""

import matplotlib.pyplot as plt
import seaborn as sns

"""ML"""

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer


pd.options.mode.chained_assignment = None 

# Limpiar datos

In [2]:
def CSV_transform(df):
    """
    Función que transforma de formato CSV a diccionario
    """
    
    df = df[~(df["lyrics"] == "error")] # No tomar en cuenta canciones que no tienen letra
    
    df["lyrics"] = df.lyrics.str.replace("\r"," ").str.replace("\n"," ").str.replace("[}{&:;,.¡!¿?\(\)\-\"\"0-9]","").str.replace("[","").str.replace("]","").str.lower() # Quitar espacios, interlineados, reemplazar algunos signos/numeros y pasar a minúsculas.
    
    df["lyrics"] = df.lyrics.apply(lambda x: unidecode(x)) # Quitar unicodes de la forma \uxxxx
    
    df["lyrics"] = df.lyrics.apply(lambda x: " ".join(x.split())) # Strippear el texto (quitar espacios innecesarios)
     
    return df

# Obtener Datos

Se tienen 3 datasets disponibles por si no se quiere/puede obtener la información personal.

In [3]:
df1 = CSV_transform(pd.read_csv("top_david_spotify.csv",usecols = ["name","lyrics"]).dropna().reset_index(drop = True))
df2 = CSV_transform(pd.read_csv("top_javier_spotify.csv",usecols = ["name","lyrics"]).dropna().reset_index(drop = True))
df3 = CSV_transform(pd.read_csv("top_jesus_spotify.csv",usecols = ["name","lyrics"]).dropna().reset_index(drop = True))

In [4]:
def Obtener_datos(numero):
    if numero == 1:
        return df1
    if numero == 2:
        return df2
    if numero == 3:
        return df3

# Obtener datos Estadísticos

In [48]:
df1_e = CSV_transform(pd.read_csv("top_david_spotify.csv",usecols = ["name","lyrics","danceability","energy","key","loudness","speechiness","acousticness","instrumentalness","liveness","valence"]).dropna().reset_index(drop = True)).drop(columns = "lyrics")
df2_e = CSV_transform(pd.read_csv("top_javier_spotify.csv",usecols = ["name","lyrics","danceability","energy","key","loudness","speechiness","acousticness","instrumentalness","liveness","valence"]).dropna().reset_index(drop = True)).drop(columns = "lyrics")
df3_e = CSV_transform(pd.read_csv("top_jesus_spotify.csv",usecols = ["name","lyrics","danceability","energy","key","loudness","speechiness","acousticness","instrumentalness","liveness","valence"]).dropna().reset_index(drop = True)).drop(columns = "lyrics")

In [49]:
def Obtener_datos_estadisticos(numero):
    if numero == 1:
        return df1_e
    if numero == 2:
        return df2_e
    if numero == 3:
        return df3_e

# Naive Bayes Classifier para Identificar idioma

En este tópico utilizo:

1. __Tema 13: clasificador de Lengua (Naïve Bayes)__
2. __Tema 4: Matriz de Incidencia (frecuencias)__

Entrenar algorítmo de clasificación para clasificar entre 17 lenguas. Se utilizará el algorítmo visto en clase, pero implementado por Sklearn.

[El conjunto de datos etiquetado](https://www.kaggle.com/datasets/basilb2s/language-detection) fue extraido de Kaggle para facilitar el etiquetado. 

1) English
2) Malayalam
3) Hindi
4) Tamil
5) Kannada
6) French
7) Spanish
8) Portuguese
9) Italian
10) Russian
11) Sweedish
12) Dutch
13) Arabic
14) Turkish
15) German
16) Danish
17) Greek

__NOTA__: Dado que la longitud de las canciones no es tan extensa, no se aplicará ningún tipo de stemming. Tampoco considero necesario aplicar la técnica de los bigramas. 



In [5]:
# Cargar Dataset
lenguajes = pd.read_csv("Language Detection.csv")

# Realizar Matriz de Incidencias

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(lenguajes.Text.str.replace("[{}:;,.¡!¿?\(\)\"\"0-9]","").to_list())

# Crear modelo 

NB = MultinomialNB() # Dejar prior como uniforme
NB.fit(X, lenguajes.Language.values)

MultinomialNB()

In [6]:
def Identificar_Idioma(df):
    X_test  = vectorizer.transform(df.lyrics)
    
    df["Idioma"] = NB.predict(X_test)
    
    return "Exitoso Identificador de Idioma"


# Completado de palabras en español en caso de contracción

En este tópico utilizo:

1. __Tema 6: Levenshtein Metric__

En español, como bien sabemos, no existen formalmente las contracciones; sin embargo, pragmáticamente se ha adquirido la costumbre de "recortar" algunas palabras y la forma de representar este fenómeno es por medio de un __'__. Comunmente estas contracciones se efectuan __en preposiciones__, esto es, stopwords. En cuanto a RI no son relevantes, pero para la interpretación literaria del texto, sí.

Se pretende identificar estas palabras y completarlas por medio de una lista de palabras comunmente contraidas (informalmente) en el español. De no detectarse alguna candidata, es mejor elimina la palabra por cuestiones de carácteres especiales.

In [7]:
def Contracciones_español(df,porcentaje = 60, lista_extra = []): 
    assert "Spanish" in df["Idioma"].unique(), 'No escuchas música en español'
    
    lista = lista_extra + [palabra for palabra in stopwords.words("Spanish") if len(palabra) >=3]
    
    español = df[df["Idioma"] == "Spanish"]["lyrics"]
    
    for indice in español.index:
            palabras_cancion = español[indice].split()
            
            for index in range(len(palabras_cancion)):
                if "'" in palabras_cancion[index]:
                    try:
                        palabras_cancion[index] =  process.extractOne(palabras_cancion[index], lista,score_cutoff = porcentaje)[0]
                    except:
                        palabras_cancion[index] = ""
                else:
                    pass
            
            cancion_corregida = " ".join(palabras_cancion)
            
            df.iloc[indice,1] = cancion_corregida
    
    return "Exitosa corrección de palabras en la lista"

# Contracciones general(')

En este tópico utilizo:

1. __Tema 3: Regex (Expresiones regulares)__

Dado que en algunas lenguas romances el uso de __'__ resulta determinante para el contexto de la oración, no puede ser fácilmente eliminado del corpus. En adición, las contracciones en el idioma inglés también existen y son muy comunes.  En general, en caso de que la contracción sea entre una preposición y una palabra relevante, es más probable que la palabra sea de mediana longitud. La función está primordialmente orientada a lenguas romance (incluyendo inglés) que las utilicen.

Se pretende identificar contracciones útiles por medio de la identificación de la longitud de la segunda palabra.

In [8]:
def Contracciones_general(df, limite = 4):
    
    
    idiomas = df[~(df["Idioma"] == "Spanish")]["lyrics"]
    
    for indice in idiomas.index:
            palabras_cancion = idiomas[indice].split()
            
            for index in range(len(palabras_cancion)):
                if "'" in palabras_cancion[index]:
                    
                    try:
                        palabras_cancion[index] =  re.search("(?<=')\w{4,}",palabras_cancion[index])[0]
                    except:
                        palabras_cancion[index] = ""
                else:
                    pass
            
            cancion_corregida = " ".join(palabras_cancion)
            
            df.iloc[indice,1] = cancion_corregida
    
    return "Exitosas correcciones generales de contracciones en la lista"

# Tokenizar Canciones

In [9]:
def Tokenizar(df):
    df["tokens"] = df["lyrics"].apply(lambda x: set(nltk.word_tokenize(x))) # Tokenizar las canciones
    
    return "Exitoso Tokenizado"

# Eliminar palabras de longitud mayor a...

In [10]:
def Eliminar_mayor_len(df,limite):
    limpias = []
    for i in range(df.shape[0]):
        limpias.append({palabra_menos for palabra_menos in df.tokens[i] if len(palabra_menos) < limite})
    
    df["tokens"] = limpias
    
    return f"Exitosa eliminación de palabras con longitud mayor a {limite}"
                        

# Eliminar palabras de longitud menor a...

In [11]:
def Eliminar_menor_len(df,limite):
    limpias = []
    for i in range(df.shape[0]):
        limpias.append({palabra_menos for palabra_menos in df.tokens[i] if len(palabra_menos) > limite})
    
    df["tokens"] = limpias
    
    return f"Exitosa eliminación de palabras con longitud menor a {limite}"

# Stopwords

In [12]:
def Stopwords(df, lista_extra = []):
    # Lista de idiomas
    idiomas = df.Idioma.unique()
    
    
    for idioma in idiomas:
        try:
            stopwords_ = stopwords.words(idioma) + lista_extra # Stopwords
        except:
            print("No hay stopwords para", idioma)
            
        canciones_idioma = df[df["Idioma"] == idioma]["tokens"]
        
        for indice in canciones_idioma.index:
            canciones_idioma[indice] = [palabra for palabra in canciones_idioma[indice] if palabra not in stopwords_]
            
        df.iloc[canciones_idioma.index,3] = canciones_idioma
        
    return "Exitosa eliminación de stopwords por idiomas identificados"
            

# WordClouds por idiomas

In [13]:
def WordClouds_idiomas(df):
    
    for idioma in df["Idioma"].unique():
    
        freq = " ".join(list(chain(*df[df["Idioma"] == idioma]["tokens"])))
        cloud = WordCloud(width = 8000,height = 8000, background_color = "black",max_words = 80).generate(freq)
    
        plt.figure(figsize=(25,15))
        plt.imshow(cloud, interpolation = "bilinear")
        plt.title(idioma, fontsize = 20, color = "black")
        plt.axis("off")
        plt.show()

# Stemmizar por idioma

En este tópico utilizo:

1. __Stemming__


La función solo está disponible para los siguientes idiomas:

1. arabic
2. danish
3. dutch
4. english
5. finnish
6. french
7. german
8. hungarian
9. italian
10. norwegian
11. portuguese
12. romanian
13. russian
14. spanish
15. swedish

De no estar disponible el idioma, simplemente se omite y se dejan los tokens originales.

In [14]:
def stemming_idiomas(df):
    
    df["tokens_stem"] = np.nan
    for idioma in df.Idioma.unique():
        
        try:
            stemmer = SnowballStemmer(idioma.lower())
            print(idioma, " ¡disponible para stemmizar!")
        except:
            continue
        
        canciones_idioma = df[df["Idioma"] == idioma]["tokens"]
        
        for indice in canciones_idioma.index:
            canciones_idioma[indice] = [stemmer.stem(palabra) for palabra in canciones_idioma[indice]]
            
        df.iloc[canciones_idioma.index,4] = canciones_idioma
        
    return "Exitosa eliminación de stopwords por idiomas identificados"

# Identificar palabras clave por (entre) canción(es)

En este tópico utilizo:

1. __Tema 8: TF-IDF__

Se busca obtener una aproximación al contexto de las canciones por medio del TF-IDF. Se pretende encontrar canciones con contextos parecidos. Se separa por idiomas para una mejor visualización, aunque ciertamente podría no hacerse y aún con eso el algortimo seguiría funcionando.

In [40]:
def palabras_mas_representativas_idioma(df, idioma):
    
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform(df[df["Idioma"] == idioma].tokens_stem.apply(lambda x: " ".join(x)).to_list())
    feature_names = vectorizer.get_feature_names()
    dense = vectors.todense()
    denselist = dense.tolist()
    tfidf = pd.DataFrame(denselist, columns=feature_names, index = df[df["Idioma"] == idioma].name)
    
    lista = {cancion: tfidf.loc[cancion,tfidf.loc[cancion,] == tfidf.loc[cancion,].max()].to_dict() for cancion in tfidf.index}
        
        
    
    return lista
    

# Clustering Estadistico

---

---

In [17]:
test = Obtener_datos(1)

In [18]:
Identificar_Idioma(test)

Contracciones_español(test)

Contracciones_general(test)

Tokenizar(test)

Eliminar_mayor_len(test,10)

Eliminar_menor_len(test,3)

Stopwords(test,["yeah","woah","ohoh","yeahyeah","eeeeheheh","oooohoooh","woohoo"])

# WordClouds_idiomas(test)

stemming_idiomas(test)

#palabras_mas_representativas_idioma(test, "English")

English  ¡disponible para stemmizar!
Spanish  ¡disponible para stemmizar!
French  ¡disponible para stemmizar!


'Exitosa eliminación de stopwords por idiomas identificados'

In [50]:
Obtener_datos_estadisticos(1)

,name,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence
0,Star Treatment,0.581,0.767,7,-5.026,0.0527,0.243000,0.001310,0.1410,0.673
1,Fireside,0.538,0.953,9,-5.611,0.0560,0.023400,0.001260,0.1130,0.740
2,Veneno,0.730,0.664,2,-6.872,0.0346,0.372000,0.000000,0.2250,0.719
3,Cuando Me Acerco A Ti,0.736,0.543,9,-9.037,0.2400,0.183000,0.000000,0.1970,0.245
4,Cornerstone,0.287,0.721,9,-5.810,0.0387,0.007640,0.000000,0.3280,0.763
5,Sur mes gardes,0.741,0.327,0,-8.684,0.0300,0.725000,0.000000,0.1010,0.590
6,ROXANNE,0.621,0.601,6,-5.616,0.1480,0.052200,0.000000,0.4600,0.457
7,Hola Señorita,0.588,0.796,0,-3.945,0.2620,0.386000,0.000000,0.1980,0.546
8,11 PM,0.777,0.712,10,-4.840,0.2770,0.217000,0.000000,0.0910,0.680
9,Chained To The Rhythm,0.562,0.800,0,-5.404,0.1120,0.081400,0.000000,0.1990,0.471


- Crear lista de terminos relacionados en los idiomas seleccionados
- Palabras relacionadas vectorialmente con las metricas spotify (despues estudiar palabras)
- Documentar los 3 documentos